This is a notebook demonstrating how to use the Python bindings for QuiZX. It will grow as more bindings are implemented for QuiZX functionality.

The bindings are implemented in the `quizx` module. They are designed such that a QuiZX implements the same interface as `pyzx.graph.BaseGraph`, so it can be passed into PyZX functions. This is handy for drawing, testing, IO, etc.

In [1]:
import quizx
import pyzx
import random
from fractions import Fraction

QuiZX has two graph implementations, but for simplicity, only `VecGraph` is exported to the Python bindings. You can construct a `VecGraph` in the usual way and build it using the same graph API as PyZX.

In [2]:
g = quizx.VecGraph()
g.add_vertex(row=0)
g.add_vertex(pyzx.VertexType.Z, row=1, phase=Fraction(1, 2))
g.add_vertex(row=2)
g.add_edge((0, 1))
g.add_edge((1, 2))
g.set_inputs((0,))
g.set_outputs((2,))

# We can tell it's a QuiZX graph by checking the backend attribute
print(g.backend)

# Since it implements the PyZX BaseGraph API, we can use PyZX for drawing
pyzx.draw(g, labels=True)

quizx-vec


In addition to explicitly calling PyZX functions, some methods in the Python bindings call PyZX under the hood. For example, the `to_matrix` method uses functions in `pyzx.tensor` to compute a numpy array, then reshapes it into a rectangular matrix from inputs to outputs.

In [3]:
g.to_matrix()

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+1.j]])

Rather than building a QuiZX graph by hand, one can be constructed using any PyZX method that takes a `backend` argument. Probably the most useful one is `Circuit.to_graph`.

In [4]:
random.seed(1337)
c = pyzx.generate.CNOT_HAD_PHASE_circuit(qubits=8, depth=150, p_t=0.03)
g = c.to_graph(backend="quizx-vec")
print(g.backend)
pyzx.draw(g)

quizx-vec


QuiZX has pure Rust implementations of the main ZX simplifiers. The ones exposed to Python are:
* `quizx.interior_clifford_simp`
* `quizx.clifford_simp`
* `quizx.fuse_gadgets`
* `quizx.full_simp`

In [5]:
g1 = g.copy()
print("Before full_simp: %d vertices, %d edges" % (g1.num_vertices(), g1.num_edges()))
quizx.full_simp(g1)
print("After full_simp:  %d vertices, %d edges" % (g1.num_vertices(), g1.num_edges()))
# pyzx.draw(g1)

Before full_simp: 281 vertices, 388 edges
After full_simp:  40 vertices, 110 edges


QuiZX also has a pure Rust implementation of the circuit extactor. After extraction, the Python bindings convert the result into a PyZX circuit and return it. QuiZX also has its own Rust data structure for circuits, but it currently doesn't have Python bindings.

In [6]:
g2 = g1.copy()
c1 = quizx.extract_circuit(g2)
pyzx.draw(c1.to_graph())
print(c1.stats())

Circuit  on 8 qubits with 126 gates.
        7 is the T-count
        119 Cliffords among which
        81 2-qubit gates (24 CNOT, 57 other) and
        32 Hadamard gates.


The other main QuiZX functionality is the stabiliser decomposer `Decomposer`. It's Python bindings are currently being overhauled, so they aren't documented here yet. Watch this space.